# THÔNG TIN NHÓM

- ĐỒ ÁN CUỐI KÌ MÔN `LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU`
- Thành viên:
    + 18120517 Nguyễn Công Bình Phương
    + 18120514 Phan Thiên Phúc

# BÀI PHÂN TÍCH

## Import các thư viện cần thiết vào môi trường

In [1]:
#import additional packages to env
!conda install -c conda-forge seaborn -y

Solving environment: done

# All requested packages already installed.



In [2]:
#import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## GIỚI THIỆU VỀ DATA

nguồn: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

- Content: This compiled dataset pulled from four other datasets linked by time and place, and was built to find signals correlated to increased suicide rates among different cohorts globally, across the socio-economic spectrum.

- References: 
    United Nations Development Program. (2018). Human development index (HDI). Retrieved from http://hdr.undp.org/en/indicators/137506

    World Bank. (2018). World development indicators: GDP (current US$) by country:1985 to 2016. Retrieved from http://databank.worldbank.org/data/source/world-development-indicators#

    [Szamil]. (2017). Suicide in the Twenty-First Century [dataset]. Retrieved from https://www.kaggle.com/szamil/suicide-in-the-twenty-first-century/notebook

    World Health Organization. (2018). Suicide prevention. Retrieved from http://www.who.int/mental_health/suicide-prevention/en/

- Inspiration: Suicide Prevention.


***Data dictionary***:
- Country: Column representing countries (string)
- Year: Column representing years (integer)
- Sex: Column representing genres (string)
- Age: Column representing ages (string)
- Suicides_no: Column representing the number of suicides (integer)
- Population: Column representing the population (integer)
- Suicides / 100k pop: Column that represents the number of suicides per 100 thousand inhabitants (float)
- Country-year: Column that represents (string)
- HDI for year: Column that represents the HDI in each year (float)
- Gdp_for_year($): Column representing GDP in each year (string)
- Gdp_per_capita($): Column representing GDP per capita (float)
- Generation: Column that represents the generation (string)

## KHÁM PHÁ DỮ LIỆU

### Đọc file dữ liệu

In [3]:
df = pd.read_csv('master.csv')
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [4]:
df.shape

(27820, 12)

### Kiểu dữ liệu của từng cột

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             27820 non-null  object 
 1   year                27820 non-null  int64  
 2   sex                 27820 non-null  object 
 3   age                 27820 non-null  object 
 4   suicides_no         27820 non-null  int64  
 5   population          27820 non-null  int64  
 6   suicides/100k pop   27820 non-null  float64
 7   country-year        27820 non-null  object 
 8   HDI for year        8364 non-null   float64
 9    gdp_for_year ($)   27820 non-null  object 
 10  gdp_per_capita ($)  27820 non-null  int64  
 11  generation          27820 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


**Nhận xét:** Các cột có kiểu dữ liệu phù hợp với dữ liệu cung cấp. Tuy nhiên, ta thấy có cột `country-year` khá thừa khi đã có cột `country` và `year` độc lập. Ta sẽ loại bỏ cột `country-year` ở trong bước tiền xử lý dữ liệu.

### Dữ liệu có các dòng bị lặp hay rỗng không?

In [6]:
df.duplicated().any()

False

=> Không có dữ liệu lặp

In [7]:
df.isnull().values.any()

True

=> Trong tập dữ liệu có các điểm dữ liệu thiếu => Ta sẽ cùng tìm hiểu kĩ hơn

In [8]:
def count_missing_values(s):
    return s.isna().sum()

def calculate_missing_percentage(s):
    return s.isna().sum() / df.shape[0] * 100

df.agg([count_missing_values, calculate_missing_percentage])

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
count_missing_values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19456.000000,0.0,0.0,0.0
calculate_missing_percentage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.935298,0.0,0.0,0.0


**Nhận xét:** Ta thấy rằng trong tập dữ liệu chỉ có duy nhất cột `HDI for year` có giá trị thiếu và  bị thiếu dữ liệu rất nhiều (~ 70%).
`HDI for year` ở đây là chỉ số `Human Development Index` của đất nước tại thời điểm được xét `year`. Tìm thiểu thêm về HDI [tại đây](https://vi.wikipedia.org/wiki/Ch%E1%BB%89_s%E1%BB%91_ph%C3%A1t_tri%E1%BB%83n_con_ng%C6%B0%E1%BB%9Di)

Vậy nên ta sẽ loại bỏ cột này ở bước tiền xử lý dữ liệu

## INSIGHT

Trước khi bước đến bước tiền xử lý dữ liệu, dựa vào những khám phá và phân tích dữ liệu cơ bản ở các bước trên. Ta sẽ chọn insight sau

***Sự ảnh hưởng của việc tự tử ở người dân đối với nền kinh tế các nước***


Dự đoán của bản thân trước khi phân tích:
```
Có thể việc tự tử ở người dân sẽ chịu ảnh hưởng và gây ảnh hưởng đến nền kinh tế của một nước. Dự đoán xu hướng của việc tự tử và sự phát triển của nền kinh tế sẽ có sự đối nghịch nhau và có mối quan hệ nhân-quả
```

## Tiền xử lý dữ liệu

Dựa theo insight nên ta sẽ tạm thời lọc ra những cột dữ liệu không cần thiết/không liên quan đến insight của ta.
Đó là các cột `sex, age, country-year, generation`. Riêng cột `HDI for year` liên quan đến insight mà ta đã chọn nhưng vì theo phân tích ở trên, ta sẽ loại bỏ vì dữ liệu thiếu quá nhiều.

### Lọc dữ liệu

In [9]:
insight_df = df.drop(['sex', 'age', 'country-year', 'generation', 'HDI for year'], axis=1)
insight_df = insight_df.rename(columns={' gdp_for_year ($) ': 'gdp_for_year ($)'})
insight_df

,country,year,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($)
0,Albania,1987,21,312900,6.71,"2,156,624,900",796
1,Albania,1987,16,308000,5.19,"2,156,624,900",796
2,Albania,1987,14,289700,4.83,"2,156,624,900",796
3,Albania,1987,1,21800,4.59,"2,156,624,900",796
4,Albania,1987,9,274300,3.28,"2,156,624,900",796
...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,107,3620833,2.96,"63,067,077,179",2309
27816,Uzbekistan,2014,9,348465,2.58,"63,067,077,179",2309
27817,Uzbekistan,2014,60,2762158,2.17,"63,067,077,179",2309
27818,Uzbekistan,2014,44,2631600,1.67,"63,067,077,179",2309


## Khám phá dữ liệu (tiếp tục) và trả lời insight

### Xem mức độ phân bố của các thuộc tính quan trọng

In [10]:
# Group attributes by country
mean_df = insight_df.drop(['year'], axis=1).groupby(['country']).mean()
mean_df

,suicides_no,population,suicides/100k pop,gdp_per_capita ($)
country,,,,
Albania,7.462121,2.360813e+05,3.502879,1859.045455
Antigua and Barbuda,0.033951,6.142679e+03,0.552901,10448.185185
Argentina,221.018817,2.784907e+06,10.469328,7914.096774
Armenia,6.392617,2.595576e+05,3.275872,1873.919463
Aruba,0.601190,7.498077e+03,9.503095,24221.642857
...,...,...,...,...
United Arab Emirates,8.638889,5.069760e+05,1.317917,42162.000000
United Kingdom,367.755376,4.674107e+06,7.502473,31908.354839
United States,2779.604839,2.165061e+07,13.819812,39269.612903


# Khám phá trên top 10 nước có số ca tự tử trung bình cao nhất và GDP cao nhất

In [11]:
top_10_suicides_mean_df = mean_df.sort_values(by=['suicides_no'], ascending=False).head(10)
top_10_suicides_mean_df

,suicides_no,population,suicides/100k pop,gdp_per_capita ($)
country,,,,
Russian Federation,3733.771605,1.139137e+07,34.892377,6518.814815
United States,2779.604839,2.165061e+07,13.819812,39269.612903
Japan,2169.091398,9.895228e+06,21.573199,36397.548387
Ukraine,952.232143,3.828777e+06,26.582321,1867.535714
Germany,933.532051,6.489986e+06,15.559904,35164.230769
France,914.241667,4.640991e+06,21.675694,31481.466667
Republic of Korea,703.575269,3.642325e+06,25.135618,14801.258065
Brazil,609.174731,1.305401e+07,5.846022,6091.483871
Poland,482.979167,3.007570e+06,15.269514,8146.458333


In [12]:
top_10_gdp_mean_df = mean_df.sort_values(by=['gdp_per_capita ($)'], ascending=False).head(10)
top_10_gdp_mean_df

,suicides_no,population,suicides/100k pop,gdp_per_capita ($)
country,,,,
Luxembourg,5.263441,3.482031e+04,16.549892,68798.387097
Qatar,3.224719,1.228682e+05,1.787360,67756.449438
Switzerland,104.035714,5.952143e+05,19.024087,62981.761905
Norway,47.200000,3.554808e+05,12.941000,57319.600000
San Marino,0.111111,2.189583e+03,4.045000,53663.666667
Denmark,57.943182,4.249326e+05,14.097159,49299.909091
United Arab Emirates,8.638889,5.069760e+05,1.317917,42162.000000
Sweden,105.572626,7.075339e+05,14.658436,41357.575419
Iceland,2.900524,2.201022e+04,12.800366,39274.753927


=> Có vẻ như trực quan hoá bằng bảng khá khó so sánh và nhận ra mối tương quan lắm

### Trực quan hoá xu hướng phát triển của kinh tế và số ca tự tử của top 20 nước

#### 10 nước top đầu về số ca tự tử

In [22]:
top_10_suicides_countries = list(top_10_suicides_mean_df.index.unique())
top_10_suicides_df = insight_df.loc[insight_df['country'].isin(top_10_suicides_countries)]
top_10_suicides_df

,country,year,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($)
4172,Brazil,1985,129,840300,15.35,"222,942,790,435",1898
4173,Brazil,1985,535,5364900,9.97,"222,942,790,435",1898
4174,Brazil,1985,1011,11986300,8.43,"222,942,790,435",1898
4175,Brazil,1985,791,10762900,7.35,"222,942,790,435",1898
4176,Brazil,1985,661,13834000,4.78,"222,942,790,435",1898
...,...,...,...,...,...,...,...
27215,United States,2015,1444,21555712,6.70,"18,120,714,000,000",60387
27216,United States,2015,1132,21633813,5.23,"18,120,714,000,000",60387
27217,United States,2015,540,11778666,4.58,"18,120,714,000,000",60387
27218,United States,2015,255,21273987,1.20,"18,120,714,000,000",60387


#### 10 nước top đầu về GDP

In [21]:
top_10_gdp_countries = list(top_10_gdp_mean_df.index.unique())
top_10_gdp_df = insight_df.loc[insight_df['country'].isin(top_10_suicides_countries)]
top_10_gdp_df

,country,year,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($)
4172,Brazil,1985,129,840300,15.35,"222,942,790,435",1898
4173,Brazil,1985,535,5364900,9.97,"222,942,790,435",1898
4174,Brazil,1985,1011,11986300,8.43,"222,942,790,435",1898
4175,Brazil,1985,791,10762900,7.35,"222,942,790,435",1898
4176,Brazil,1985,661,13834000,4.78,"222,942,790,435",1898
...,...,...,...,...,...,...,...
27215,United States,2015,1444,21555712,6.70,"18,120,714,000,000",60387
27216,United States,2015,1132,21633813,5.23,"18,120,714,000,000",60387
27217,United States,2015,540,11778666,4.58,"18,120,714,000,000",60387
27218,United States,2015,255,21273987,1.20,"18,120,714,000,000",60387


# Nhận xét

Giống như nhận định mà ta đã đưa ra trước khi tìm hiểu và trả lời câu hỏi

```
Việc tự tử ở người dân sẽ chịu ảnh hưởng và gây ảnh hưởng đến nền kinh tế của một nước. Xu hướng của việc tự tử và sự phát triển của nền kinh tế sẽ có sự đối nghịch nhau và có mối quan hệ nhân-quả
```
Điều này đã được ta thể hiện ở `top 10 quốc gia có số người tự tử trung bình cao nhất` và `top 10 quốc gia có GDP trung bình cao nhất` mà ta thu thập được

Có thể việc này bắt nguồn từ việc các chính sách thúc đẩy gia tăng kinh tế của các nước nhưng các chính sách cân bằng dân số và nhập/xuất cư lại không hợp lý khiến cho việc người lao động chịu áp lực công việc ngày một tăng dẫn đến tỉ lệ tự tử tăng.

Tuy việc này hầu hết đã được tính toán và cân bằng ở các chính sách điều chỉnh dân số nhưng có thể sự tăng/giảm đột ngột ở tỉ lệ tự tử này sẽ là một điều mà các nhà lãnh đạo cần để ý khi thực hiện các chính sách về kinh tế và lao động